In [ ]:
import tensorflow as tf
tf.random.set_seed(42)
import numpy as np
np.random.seed(42)
import time
import pickle

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import plotly.offline as pyo
from plotly import subplots
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.tsa.api as smt

from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense,Input,concatenate
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import lightgbm as lgb

from datetime import datetime, timedelta
from sklearn.model_selection import KFold

from baseFunctions import *
from data_helpers import processData6, featureEngineering, getSequencesFast, removeOutliers, create_sequences, processAllData

import dask.dataframe as dd
import plotly.express as px


In [ ]:
data, propDicts, flippedPropDicts = processData6()
data, timeFeatures = featureEngineering(data,splits=[2,2,2,2])

In [ ]:
initial_lag = 16 # 16 = independent from previous predictions
targetLags = [1,2,3,4,5,6,7,8,9,10]
featureLags = [1,2,3,4,5,6,7,8,9,10]
rolling = [7,21]

data, addedF = processAllData(data, targetLags, featureLags, rolling, initial_lag)

In [ ]:
def calcLossArima(pred, y, logTransform):
    if logTransform:
        a = np.exp(pred) -1
        y = np.exp(y) -1 
    else:
        a = (pred)
        y = (y)

    if (a < 0).any():
        a = np.clip(a, 0, 1e20)

    rmsleTrain = np.sqrt(mean_squared_log_error(a,y))
    return rmsleTrain

def calcLossLGBM(pred, y):
    logPred = np.reshape(pred, (pred.shape[0],1))
    a = np.exp(logPred) -1

    logy = np.reshape(y, (pred.shape[0],1))
    y = np.exp(logy) -1

    rmsleTrain = np.sqrt(np.mean((a-y)**2))
    return rmsleTrain
def mse(pred,y):
    return np.sqrt(np.mean((pred-y)**2))
def calcLossLGBMArima1(pred, sales, arima, predictDiff):
    if predictDiff:
        pred = np.reshape(pred, (pred.shape[0],1)) + np.reshape(arima, (pred.shape[0],1))
    a = np.exp(pred) -1

    if (a < 0).any():
        a = np.clip(a, 0, 1e20)
    rmsleTrain = np.sqrt(mean_squared_log_error(a,sales))
    return rmsleTrain
def calcLossLGBMArima2(pred, y, arima, predictDiff):
    if predictDiff:
        pred = np.reshape(pred, (pred.shape[0],1)) + np.reshape(arima, (pred.shape[0],1))
        y = np.reshape(y, (pred.shape[0],1)) + np.reshape(arima, (pred.shape[0],1))
    a = np.exp(pred) -1
    y = np.exp(y) -1

    if (a < 0).any():
        a = np.clip(a, 0, 1e20)
    rmsleTrain = np.sqrt(mean_squared_log_error(a,y))
    return rmsleTrain
def plotLGBM(i, logTransform, salesDomain, pred, y, base):
    if salesDomain:
        pred = np.reshape(pred, (pred.shape[0],1)) + np.reshape(base, (pred.shape[0],1))
        y = np.reshape(y, (pred.shape[0],1)) + np.reshape(base, (pred.shape[0],1))
    if logTransform:
        a = np.exp(pred[i:i+16])-1
        y = np.exp(y[i:i+16]) -1
        arima = np.exp(base[i:i+16]) - 1
    else:
        a = (pred[i:i+16])
        y = (y[i:i+16])
        arima = base[i:i+16]
    x = range(len(a))   

    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
    axs.plot(x, y, color='blue',label='Original')
    axs.plot(x, a, color='red',label='pred')
    axs.plot(x, arima, color='orange',label='arima')
    axs.set_title('index: '+str(i))
    fig.subplots_adjust(hspace=0.5)


In [ ]:
""" define features and test/train partition """
trainF = [
       'store_nbr', 'family', 
       #'sales', 
       'onpromotion',# 'dataT',
       'city', 'state', 'type', 'cluster', 
       'dcoilwtico', 
       'holidayType',
       'description', 
       'transferred', 
       #'transactions', 
       'store_closed']
timeF = [
       'linear_time', 'day_of_year', 'day_of_year_f12_0', 'day_of_year_f104_0','day_of_year_f24_0',  'day_of_year_f52_0',
       'day_of_year_f12_180', 'day_of_year_f104_180','day_of_year_f24_180','day_of_year_f52_180', 
       'weekday', 'month'
       ]

allF = addedF + trainF + timeF

data['predTarget'] = np.nan

for family in data.family.unique():
       date_string_test = "2017-08-01"    
       train_bigDf = data.loc[(data.date < date_string_test)  & (data.dataT =='train') & (data.family ==family)] #& (data.date > "2015-05-01")
       test_bigDf  = data.loc[(data.date >= date_string_test) & (data.dataT !='test') & (data.family ==family)]
       val_bigDf   = data.loc[ (data.dataT =='test') & (data.family ==family)]     

       predictDiff16 = True
       targetF, refF = 'target', 'ref'
       logTransform16 = True

       X_train16, X_test16, X_pred16 = train_bigDf[allF],test_bigDf[allF],val_bigDf[allF]
       y_train16, y_test16 = train_bigDf[targetF], test_bigDf[targetF]
       baseTrain16, baseTest16, basePred16 = train_bigDf[refF], test_bigDf[refF],val_bigDf[refF]


       # compare against base lgbm that just predicts always t+16
       sample_weights = (train_bigDf.date > "2017-06-01") + 1#train_bigDf.logSales+1 #np.reshape(( * train_bigDf.onpromotion.std()) + train_bigDf.onpromotion.mean()+ 1, (-1,1))
       sample_weights = sample_weights/ max(sample_weights)


       # Set parameters for LGBM model
       params = {
           'boosting':'gbdt',#'gbdt', #'rf' #'dart'
           'objective': 'regression',  # Assuming you're doing regression
           'metric': 'mse',  # Mean squared error
           'num_leaves': 7,
           'feature_fraction': 0.9,
           'bagging_fraction': 0.8,
           'bagging_freq': 5,
           'verbose': -1,
           'force_col_wise':True,
       }   

       # Train the model
       gbm = lgb.train(params, lgb.Dataset(X_train16, label=y_train16
             # , weight=sample_weights
              ), 2000,valid_sets=[lgb.Dataset(X_test16, label=y_test16)]#,num_boost_round=100
       ,callbacks=[lgb.early_stopping(stopping_rounds=1000)]
       )  
       predtrainLGBM = gbm.predict(X_train16)
       predtestLGBM = gbm.predict(X_test16)
       predvalLGBM = gbm.predict(X_pred16)
       print('---------------',family,'--------------------')
       #print('arima errors: ', calcLossArima(train_subDf.ref, train_subDf.logSales, logTransform), calcLossArima(test_subDf.ref, test_subDf.logSales, logTransform))#, calcLossArima(val_subDf.ref, val_subDf.logSales, logTransform, baseTest))
       print('arima errors: ', calcLossArima(train_bigDf.ref, train_bigDf.logSales, logTransform16), calcLossArima(test_bigDf.ref, test_bigDf.logSales, logTransform16))
       #print('lgbm errors: ', calcLossLGBM(predtrainLGBM, y_train), calcLossLGBM(predtestLGBM, y_test))#, calcLossLGBM(predvalLGBM, y_val))

       print('sales errors: ', mse(predtrainLGBM, y_train16), mse(predtestLGBM, y_test16))
       print('sales errors: ', calcLossLGBMArima2(predtrainLGBM, y_train16, baseTrain16, predictDiff16), calcLossLGBMArima2(predtestLGBM, y_test16, baseTest16,predictDiff16))#, calcLossLGBMArima2(predvalLGBM, y_val,baseVal))
       #print('sales errors: ', calcLossLGBMArima1(predtrainLGBM, train_subDf.sales, baseTrain), calcLossLGBMArima1(predtestLGBM, test_subDf.sales, baseTest), calcLossLGBMArima1(predvalLGBM, val_subDf.sales,baseVal))

       data.loc[train_bigDf.index,['predTarget']] = predtrainLGBM
       data.loc[test_bigDf.index,['predTarget']] = predtestLGBM
       data.loc[val_bigDf.index,['predTarget']] = predvalLGBM

# 0.399 test score


In [ ]:
data['logError'] = (data['predTarget'] + data['ref'] - data['logSales'])**2

In [ ]:
date_string_test = "2017-08-01"    
train_bigDf = data.loc[(data.date < date_string_test) & (data.date > "2015-05-01") & (data.dataT =='train')]
test_bigDf  = data.loc[(data.date >= date_string_test) & (data.dataT !='test')]
val_bigDf   = data.loc[ (data.dataT =='test')]

np.sqrt(train_bigDf['logError'].mean()),np.sqrt(test_bigDf['logError'].mean())

# (0.39592463725045063, 0.39190685821584076)   10 leaves
# (0.40743601263950485, 0.39219064204608006)    5 leaves
# (0.3744699019480408, 0.39237606632021116)    15 leaves

# 0.38728236949929384, 0.3842126947865215 more features - no weight, vs 0.38269330293604925 0.38686356026792235 for general model
# 0.38441747955591077, 0.38421150630874995 more feat, same as above, but more data (all past)
# 0.38356193996058435, 0.38461630057313745 same above, but 7 trees -> 0.409 score

# 0.36132912353473373, 0.3494057402494539 with cross validation & last data in train

### cross validation pred

In [ ]:
data['predTarget'] = np.nan
print('watchout "test" data is included in training')

for family in data.family.unique():
    params = {
               'boosting':'gbdt',#'gbdt', #'rf' #'dart'
               'objective': 'regression',  # Assuming you're doing regression
               'metric': 'mse',  # Mean squared error
               'num_leaves': 10,
               'feature_fraction': 0.9,
               'bagging_fraction': 0.8,
               'bagging_freq': 5,
               'verbose': -1,
               'force_col_wise':True,

        #       colsample_bytree=0.7,
        #learning_rate=0.055,
        #min_child_samples=10,
        #num_leaves=19,
           }   
    
    date_string_test = "2017-08-01"   
    train_bigDf = data.loc[(data.dataT =='train') & (data.family ==family)] #& (data.date > "2015-05-01")
    test_bigDf  = data.loc[(data.date >= date_string_test) & (data.dataT !='test') & (data.family ==family)] # just for validation
    val_bigDf   = data.loc[ (data.dataT =='test') & (data.family ==family)]   
    
    predictDiff16 = True
    targetF, refF = 'target', 'ref'
    logTransform16 = True
    X_train16, X_test16, X_pred16 = train_bigDf[allF],test_bigDf[allF],val_bigDf[allF]
    y_train16, y_test16 = train_bigDf[targetF], test_bigDf[targetF]
    baseTrain16, baseTest16, basePred16 = train_bigDf[refF], test_bigDf[refF],val_bigDf[refF]
    
    nsplits=5
    num_iter=2000
    
    kf = KFold(n_splits=nsplits, shuffle=False)# , random_state=42) #random doesn't help
    splits = kf.split(X_train16,y_train16)
    
    cv_results = lgb.cv(
        params,
        lgb.Dataset(X_train16, label=y_train16
        #, weight=sample_weights
        ),
        num_boost_round=num_iter,
        folds=splits,
        stratified=False,  # Set to True for stratified sampling in classification
        #early_stopping_rounds=50,  # Stop if score doesn't improve for 50 rounds
        metrics=['mse'],  # Evaluation metrics to track
        seed=42,  # Set a seed for reproducibility
        return_cvbooster=True
        ,callbacks=[lgb.early_stopping(stopping_rounds=1000)]
    )
    
    lenIter = len(cv_results['valid l2-mean'])
    print('---------------',family,'--------------------')
    print(cv_results['valid l2-mean'][lenIter-1], cv_results['valid l2-stdv'][lenIter-1])
    trainLGBM, testLGBM, predLGBM = [],[],[]
    init=False
    for i in range(nsplits):
        predtrainLGBM = cv_results['cvbooster'].boosters[i].predict(X_train16)
        predtestLGBM = cv_results['cvbooster'].boosters[i].predict(X_test16)
        predvalLGBM = cv_results['cvbooster'].boosters[i].predict(X_pred16)
        if init:
            trainLGBM = trainLGBM + predtrainLGBM
            testLGBM = testLGBM + predtestLGBM
            predLGBM = predLGBM + predvalLGBM
        else:
            trainLGBM = predtrainLGBM
            testLGBM = predtestLGBM
            predLGBM = predvalLGBM
            init=True
    
        print('sales errors: ', calcLossLGBMArima2(predtrainLGBM, y_train16, baseTrain16, predictDiff16), calcLossLGBMArima2(predtestLGBM, y_test16, baseTest16,predictDiff16))
    
    trainLGBM = trainLGBM/nsplits
    testLGBM = testLGBM/nsplits
    predLGBM = predLGBM/nsplits
    print('overall',calcLossLGBMArima2(trainLGBM, y_train16, baseTrain16, predictDiff16), calcLossLGBMArima2(testLGBM, y_test16, baseTest16,predictDiff16))
    data.loc[train_bigDf.index,['predTarget']] = trainLGBM
    data.loc[test_bigDf.index,['predTarget']] = testLGBM
    data.loc[val_bigDf.index,['predTarget']] = predLGBM

In [ ]:
#overall 0.5828023480900187 0.6080170446516043 7treest

# LSTM per family 

In [ ]:
# use a bidirectional lstm per family  https://www.kaggle.com/code/markbquant/jul-21-timeseries-windows-rnn
 
branch[i] = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(wsize, return_sequences=True))(inp)
branch[i] = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(wsize))(branch[i])
branch[i] = tf.keras.layers.Dense(wsize, activation="relu")(branch[i])
branch[i] = tf.keras.layers.Dense(wsize//2, activation="relu")(branch[i])
branch[i] = tf.keras.layers.Dense(1, activation="relu")(branch[i])

# analyze residuals

In [ ]:
def plotDistOfTestTrain(feat, test, train,labelA ='test',labelB='train'):
    a = test[[feat]].value_counts()
    b = train[[feat]].value_counts()

    indexa, indexb = [],[]
    for i, val in enumerate(a.index):
        indexa.append(val[0])
    for i, val in enumerate(b.index):
        indexb.append(val[0])

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(indexa, a.values/np.sum(a), alpha=0.5, label=labelA)
    ax.bar(indexb, b.values/np.sum(b), alpha=0.5, label=labelB)
    ax.set_title('Distribution of' + feat)
    ax.set_xlabel('Category')
    ax.set_ylabel('Frequency')
    plt.xticks(rotation=90)
    ax.legend()
    plt.show()

def display2Ddifferences(feata, featb, dfa, dfb):
    a = dfa[[feata,featb]].value_counts().sort_index()
    b = dfb[[feata,featb]].value_counts().sort_index()
    xa, xb = [],[]
    ya, yb = [],[]
    for i, val in enumerate(a.index):
        xa.append(val[0])
        ya.append(val[1])
    for i, val in enumerate(b.index):
        xb.append(val[0])
        yb.append(val[1])

    df1 = pd.DataFrame()
    df1[feata] = xa
    df1[featb]=ya
    df1['f'] = a.values/np.sum(a)*100

    df2 = pd.DataFrame()
    df2[feata] = xb
    df2[featb]=yb
    df2['f'] = b.values/np.sum(b)*100


    fig = px.scatter(df1,
        y=feata,
    x=featb,
        color='f',title='bad'
    )
    fig.show()

    fig1 = px.scatter(df2,
        y=feata,
        x=featb,
        color='f',title='good'
    )
    fig1.show()

    return df1, df2


In [ ]:
train_bigDf = data.loc[(data.date < date_string_test) & (data.date > "2015-05-01") & (data.dataT =='train')]
test_bigDf  = data.loc[(data.date >= date_string_test) & (data.dataT !='test')]
val_bigDf   = data.loc[ (data.dataT =='test')]

train_bigDf['err_predTarget'] = np.abs(train_bigDf['predTarget'] - train_bigDf['target'])
test_bigDf['err_predTarget'] = np.abs(test_bigDf['predTarget'] - test_bigDf['target'])



train_bigDf['predLogSales'] = train_bigDf['predTarget']+train_bigDf['salesArima']
test_bigDf['predLogSales'] = test_bigDf['predTarget']+test_bigDf['salesArima']
train_bigDf['err_LogSales'] = (train_bigDf['predLogSales'] - train_bigDf['logSales'])**2
test_bigDf['err_LogSales'] =(test_bigDf['predLogSales'] - test_bigDf['logSales'])**2
train_bigDf['err_arimaLogSales'] = (train_bigDf['salesArima'] - train_bigDf['logSales'])**2
test_bigDf['err_arimaLogSales'] = (test_bigDf['salesArima'] - test_bigDf['logSales'])**2

# arimaLogError > errLogSales = positive (= reduction of errLogSales)
train_bigDf['improvedScoreByLGBM'] = train_bigDf['err_arimaLogSales'] - train_bigDf['err_LogSales']
test_bigDf['improvedScoreByLGBM'] = test_bigDf['err_arimaLogSales'] - test_bigDf['err_LogSales']

train_bigDf['predSales'] = np.exp(train_bigDf.predLogSales) -1
test_bigDf['predSales'] = np.exp(test_bigDf.predLogSales) -1#
train_bigDf['err_sales'] = np.abs(train_bigDf['predSales'] - train_bigDf['sales'])
test_bigDf['err_sales'] = np.abs(test_bigDf['predSales'] - test_bigDf['sales'])

train_bigDf['predArimaSales'] = np.exp(train_bigDf.salesArima) -1
test_bigDf['predArimaSales'] = np.exp(test_bigDf.salesArima) -1
train_bigDf['err_salesArima'] = np.abs(train_bigDf['predArimaSales'] - train_bigDf['sales'])
test_bigDf['err_salesArima'] = np.abs(test_bigDf['predArimaSales'] - test_bigDf['sales'])

In [ ]:
val_bigDf['sales'] = np.exp(val_bigDf.predTarget + val_bigDf.salesArima)-1

val_bigDf[['id','sales']].set_index('id').to_csv('arima_lgbm_perFam_crossVal_10trees.csv')

In [ ]:
badPredictionsTest = test_bigDf.loc[test_bigDf.improvedScoreByLGBM>0]
goodPredictionsTest = test_bigDf.loc[test_bigDf.improvedScoreByLGBM<0]
badPredictionsTrain = train_bigDf.loc[train_bigDf.improvedScoreByLGBM>0]
goodPredictionsTrain = train_bigDf.loc[train_bigDf.improvedScoreByLGBM<0]

plotDistOfTestTrain('family',badPredictionsTest,goodPredictionsTest,'bad','good')
plotDistOfTestTrain('store_nbr',badPredictionsTest,goodPredictionsTest,'bad','good')

In [ ]:
badPredictionsTest = test_bigDf.loc[test_bigDf.err_LogSales>1]
goodPredictionsTest = test_bigDf.loc[test_bigDf.err_LogSales<1]
badPredictionsTrain = train_bigDf.loc[train_bigDf.err_LogSales>1]
goodPredictionsTrain = train_bigDf.loc[train_bigDf.err_LogSales<1]

badFreq, goodFreq = display2Ddifferences('store_nbr','family', badPredictionsTest, goodPredictionsTest)
badFreq, goodFreq = display2Ddifferences('store_nbr','family', badPredictionsTrain, goodPredictionsTrain)